In [1]:
import os
import json
import math
from datetime import datetime, timedelta
from typing import Optional, Union

import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point


In [2]:
root_path = os.path.abspath('..')
bounding_file = os.path.join(root_path, 'data/boundary.geojson')

In [3]:
bound_gdf = gpd.read_file(bounding_file, driver='GeoJSON')

In [30]:

def get_stations_from_bbox():
    """Return a list of stations IDs found within a bounding box.
    Returns:
        GeoDataFrame[str]: A geodataframe with all stations' coordinates, id, and name
    """
    
    data_url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json"
    response = requests.get(data_url)
    json_dict = response.json()
    
    stations = []
    for station in json_dict["stations"]:
        station_info = []
        station_info.append(station['name'])
        station_info.append(station['id'])
        station_info.append(station['lat'])
        station_info.append(station['lng'])
        
        stations.append(station_info)
    
    df = pd.DataFrame(stations, columns=['name', 'id', 'lat', 'lng'])
    geometry = [Point(xy) for xy in zip(df.lng, df.lat)]
    points_gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

    return points_gdf

In [31]:
points = get_stations_from_bbox()

In [50]:
sub_stations = points[points.within(bound_gdf.loc[0, 'geometry'])]

,name,id,lat,lng,geometry
53,Ocean City Inlet,8570283,38.328267,-75.091086,POINT (-75.09109 38.32827)
54,Bishops Head,8571421,38.220000,-76.038333,POINT (-76.03833 38.22000)
55,Cambridge,8571892,38.572500,-76.061667,POINT (-76.06167 38.57250)
60,Solomons Island,8577330,38.317222,-76.450833,POINT (-76.45083 38.31722)
62,Wachapreague,8631044,37.607778,-75.685833,POINT (-75.68583 37.60778)
63,Kiptopeke,8632200,37.165190,-75.988444,POINT (-75.98844 37.16519)
64,Dahlgren,8635027,38.319753,-77.036597,POINT (-77.03660 38.31975)
65,Lewisetta,8635750,37.996389,-76.465556,POINT (-76.46556 37.99639)
66,Windmill Point,8636580,37.615500,-76.289778,POINT (-76.28978 37.61550)
67,Yorktown USCG Training Center,8637689,37.226500,-76.478806,POINT (-76.47881 37.22650)
